# Evaluation
Bisher hatten Sie ausschließlich die Möglichkeit, die Ergebnisse des Clusterings optisch zu bewerten. In diesem Abschnitt werden deshalb Kriterien vorgestellt, die zur Bewertung der Güte eines Clusterings herangezogen werden können.

Importieren Sie zunächst die benötigten Bibliotheken und laden Sie die bereits bekannten Datensätze.

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

from tui_dsmt.clustering import interactive_td2, td2_graph, silhouette_graph
from tui_dsmt.clustering.datasets import clustering_example1, clustering_example2

## Inhaltsverzeichnis
- [Kompaktheit](#Kompaktheit)
- [Silhouetten](#Silhouetten)

## Kompaktheit
Kompaktheit ist ein Maß für die Güte eines Clusterings mit Zentroiden, wie sie beispielsweise bei k-Means verwendet werden.

Um die Kompaktheit eines Clusters $C$ zu ermitteln, wird die Summe der quadratischen Distanz zum Zentroid $\overline{x}_C$ gebildet. $dist$ steht dabei für eine beliebige Distanzfunktion. In den bisherigen Beispielen kam ausschließlich die euklidische Distanz zur Anwendung.

$$
TD^2(C) = \sum_{p \in C} dist(p, \overline{x}_C)^2
$$

Zur Berechnung der Kompaktheit eines Clusterings mit $k$ Clustern werden die Ergebnisse aller Cluster aufsummiert.

$$
TD^2 = \sum_{i=1}^{k} TD^2(C_i)
$$

In Python lässt sich $TD^2$ wie folgt ausdrücken. (`df` wird dabei um zwei Spalten `cx` und `cy` für die Koordinaten des zugehörigen Clusterzentrums erweitert. Auf Basis dieser wird anschließend die quadrierte euklidische Distanz berechnet und aufsummiert.)

In [ ]:
def TD2(df, centroids):
    cx = df['cluster'].map(lambda i: centroids[i][0])
    cy = df['cluster'].map(lambda i: centroids[i][1])

    return sum((df['x'] - cx) ** 2 + (df['y'] - cy) ** 2)

Die folgende Zelle gibt Ihnen die Möglichkeit, erneut mit der Anzahl der Clusterzentren zu experimentieren und dabei den Kompaktheitswert zu beobachten.

In [ ]:
interactive_td2(clustering_example1)

Wie Sie feststellen können, sinkt die Kompaktheit monoton (nicht streng) mit einer zunehmenden Anzahl an Clustern. Steigt die Clusteranzahl bis zur Anzahl der Punkte, liegt die Kompaktheit sogar bei $0$.

Insbesondere in der logarithmischen Darstellung der Kompaktheit in Abhängigkeit der Clusteranzahl können Sie dennoch ein Muster ausmachen, das Ihnen potentiell bei der Suche nach einer geeigneten Clusteranzahl helfen kann. Nach Überschreiten der optimalen Clusteranzahl scheint die Kompaktheit deutlich langsamer zu sinken als zuvor.

In [ ]:
td2_graph(clustering_example1)

Dieses Verhalten lässt sich allerdings nicht bei Datensätzen beobachten, die sich nicht mit dem k-Means-Algorithmus einteilen lassen, wie beispielsweise den Ihnen bereits bekannten ineinandergreifenden Spiralen.

In [ ]:
td2_graph(clustering_example2)

## Silhouetten
Der sogenannte Silhouettenkoeffizient gibt als Kennzahl an, wie gut die Zuordnung der Punkte zu den jeweils zwei nächstgelegenen Clustern ist. Es fließt also ein, wie nah ein Objekt am eigenen Cluster und wie entfernt das Objekt vom nächsten Cluster liegt. Der Wert ist dabei unabhängig von der Clusteranzahl und kann dementsprechend auch für Cluster berechnet werden, die beispielsweise mit DBScan erzeugt wurden.

Nun sei $o$ ein Punkt in den Daten, $a(o)$ die durchschnittliche Distanz zu anderen Punkten des zugehörigen Cluster und $b(o)$ die durchschnittliche Distanz zu Punkten des nächstgelegenen Clusters, dem $o$ nicht angehört. Die Silhouette des Objekts $o$ berechnet sich dann wie folgt:

$$
s(o) = \begin{cases}
0                                    & \text{falls $o$ das einzige Element in seinem Cluster ist} \\
\dfrac{b(o) - a(o)}{max(a(o), b(o))} & \text{sonst}
\end{cases}
$$

Es ergeben sich Werte im Intervall $[-1, 1]$. Dabei stehen Werte nahe $1$ für eine passende Zuordnung, während Werte um $0$  eine uneindeutige und Werte nahe $-1$ eine schlechte Zuordnung repräsentieren.

Um den Silhouettenkoeffizient für ein vollständiges Clustering zu berechnen, werden die Werte aller Objekte aller Cluster aufsummiert und durch die Anzahl der Objekte dividiert:

$$
S(C_M) = \dfrac{\sum_{C \in C_M} \sum_{o \in C} s(o)}{|O|}
$$

Je höher das Ergebnis ist, desto besser ist das Clustering. So spricht man bei Werten im Intervall
- $(0.7, 1.0]$ von einer starken,
- $(0.5, 0.7]$ von einer brauchbaren,
- $(0.25, 0.5]$ von einer schwachen und
- $[0, 0.25]$ von keiner Struktur.

Wie üblich lässt sich diese Berechnung auch mit Hilfe von Pandas in Python abbilden...

In [ ]:
def s(df, o):
    # Spalte d enthält die Distanz jedes Objekts zum Objekt o
    df['d'] = (df['x'] - o['x']) ** 2 + (df['y'] - o['y']) ** 2

    # durchschnittlicher Abstand zum eigenen Cluster
    o_cluster = df[df['cluster'] == o['cluster']]
    a = o_cluster['d'].sum() / (len(o_cluster) - 1)

    # durchschnittlicher Abstand zum nächstgelegenen Cluster
    other_clusters = df[df['cluster'] != o['cluster']]
    min_dist = other_clusters.groupby('cluster')['d'].sum().nsmallest(1)

    next_cluster_id = min_dist.index[0]
    next_cluster_d, = min_dist
    next_cluster_size = len(df[df['cluster'] == next_cluster_id])

    b = next_cluster_d / next_cluster_size

    # Berechnung von s(o)
    return (b - a) / max(a, b)


def S(df):
    df['s'] = df.apply(lambda o: s(df, o), axis=1)
    return df['s'].sum() / len(df)

... und anschließend nutzen, um beispielsweise die beste Anzahl an Clustern für den k-Means Algorithmus ausfindig zu machen. Beachten Sie dabei, dass für den ersten Datensatz ein Maximum bei $k=15$ zu erkennen ist. (Da die zufällige Initialisierung bei k-Means das Ergebnis verändern kann, wird hier ein fixer Wert für die Initialisierung des Zufallszahlengenerators verwendet.)

In [ ]:
silhouette_graph(clustering_example1, random_state=1)

Für den zweiten, spiralförmigen Datensatz dagegen scheint der Koeffizient mit der Anzahl der Clusterzentren zu wachsen. Er erreicht dabei aber nie den Wert einer starken Struktur, was ein Indiz gegen die Anwendbarkeit des k-Means Algorithmus ist.

In [ ]:
silhouette_graph(clustering_example2, random_state=1)

Natürlich existiert auch für den Silhouetten-Koeffizient eine effiziente Implementierung in `scikit-learn`. Auch wenn die Ergebnisse auf Grund der komplexeren Implementierung geringfügig abweichen, können Sie dennoch mit Leichtigkeit die optimale Clusteranzahl ablesen.

In [ ]:
from sklearn.metrics import silhouette_score

px.bar(pd.DataFrame({
    'k': range(2, 30),
    'S': (silhouette_score(clustering_example1, KMeans(k, random_state=1).fit_predict(clustering_example1))
          for k in range(2, 30))
}), x='k', y='S')